<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/AlgoTradingTest01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sourcing Alpaca Credentials

In [2]:
credentials = open("/content/drive/MyDrive/Colab Notebooks/alpaca.txt","r")
lines = credentials.readlines()
consumer_key = lines[0].rstrip().replace("alpaca_key=","")
consumer_secret = lines[1].rstrip().replace("alpaca_secret=","")
#print(consumer_secret)


In [3]:
!pip install alpaca-trade-api requests beautifulsoup4 transformers

In [4]:
import pandas as pd
from alpaca_trade_api import REST, Stream, TimeFrame,TimeFrameUnit
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta

API_KEY = consumer_key
API_SECRET = consumer_secret

def fetch_news_summary(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        summary = soup.find('p').text
        return summary.strip()
    except Exception as e:
        print(f"Error fetching news summary: {e}")
        return None

def get_news_data(ALPACA_API_KEY,ALPACA_SECRET_KEY,ticker):
    # Get News for last 24 hrs
    today = datetime.today() - timedelta(1)
    yesterday = today - timedelta(15)
    today = today.strftime('%Y-%m-%d')
    yesterday = yesterday.strftime('%Y-%m-%d')

    # Initialize Alpaca API
    # Get the news data
    rest_client = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    news_list = rest_client.get_news(ticker,yesterday, today,limit=500)
    rest_client2 = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    bars = rest_client2.get_bars(ticker, TimeFrame(59, TimeFrameUnit.Minute), yesterday,today, adjustment='raw').df
    bars['time'] = bars.index.strftime('%Y-%m-%d %H:%M')
    bars['Ticker'] = ticker

    # Convert the news data into a pandas DataFrame
    data = {
        'Title': [],
        'time': [],
        'Source': [],
        'Url': [],
        'Summary': [],
        'Ticker' : []
    }

    for news in news_list:
        data['Title'].append(news.headline)
        data['time'].append(news.created_at.strftime('%Y-%m-%d %H:%M'))
        data['Source'].append(news.source)
        data['Url'].append(news.url)
        data['Summary'].append(fetch_news_summary(news.url))
        data['Ticker'].append(ticker)

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    bars["time"] = pd.to_datetime(bars["time"])

    return df,bars

if __name__ == "__main__":
    news_df,bars_data = get_news_data(API_KEY,API_SECRET,"TSLA")

In [5]:
news_df

,Title,time,Source,Url,Summary,Ticker
0,AI Frenzy Result – Perma Bulls Revered And Per...,2023-07-31 15:01:00,benzinga,https://www.benzinga.com/23/07/33472022/ai-fre...,"To gain an edge, this is what you need to know...",TSLA
1,"Exxon In Talks With Tesla, Ford, VW On Supplyi...",2023-07-31 17:47:00,benzinga,https://www.benzinga.com/news/23/07/33476063/e...,Never miss a trade again with the fastest news...,TSLA
2,10 Consumer Discretionary Stocks Whale Activit...,2023-07-31 17:35:00,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,TSLA
3,'Uber To Purchase 100 Tesla Model Ys For The J...,2023-07-31 17:18:00,benzinga,https://www.benzinga.com/news/23/07/33475622/u...,Never miss a trade again with the fastest news...,TSLA
4,Last Week's EV Updates Were All About Electric...,2023-07-31 15:24:00,benzinga,https://www.benzinga.com/23/07/33472770/last-w...,"Last week, Tesla Inc TSLA knocked on India’s d...",TSLA
...,...,...,...,...,...,...
303,"Elon Musk Poses 'Top Risk' To Tesla, Says EV E...",2023-07-17 09:27:00,benzinga,https://www.benzinga.com/news/23/07/33243754/e...,As the electric vehicle market continues to gr...,TSLA
304,Why Tesla Will Likely Stay Quiet About Model 3...,2023-07-17 06:31:00,benzinga,https://www.benzinga.com/analyst-ratings/analy...,"Tesla, Inc. TSLA is scheduled to host its seco...",TSLA
305,Tesla Supercharges Referral Program With Expan...,2023-07-17 03:20:00,benzinga,https://www.benzinga.com/news/23/07/33242219/t...,EV giant Tesla Inc TSLA has tweaked its referr...,TSLA
306,Cathie Wood's ARK Losing Its Shine? Flagship F...,2023-07-17 01:42:00,benzinga,https://www.benzinga.com/news/23/07/33241982/c...,Investors have reportedly been pulling money o...,TSLA


In [ ]:
from transformers import pipeline,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch

def news_sentiment_handler(news):

      df_array = np.array(news[["Title"]])
      df_list = list(df_array[:,0])

      tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
      model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
      inputs = tokenizer(df_list, padding = True, truncation = True, return_tensors='pt') #tokenize text to be sent to model
      outputs = model(**inputs)
      predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

      model.config.id2label
      positive = predictions[:, 0].tolist()
      negative = predictions[:, 1].tolist()
      neutral = predictions[:, 2].tolist()
      table = {'Headline':df_list, "Positive":positive, "Negative":negative, "Neutral":neutral}
      sentiment_data = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

      return(sentiment_data)

sentiment_df = news_sentiment_handler(news_df)


In [105]:
merged_dataframe = pd.merge_asof(bars_data,news_df.sort_values('time'), on="time", by="Ticker",
                                 tolerance=pd.Timedelta("10m"))


Title      100
time       100
Source     100
Url        100
Summary    100
Ticker     100
dtype: int64